## Demo

In [ ]:
from wavescapes import *
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from utils import *
import pandas as pd
import json

In [ ]:
scores_muse = glob('./MS3/*.mscx') #to understand how to make musescores work

In [ ]:
json_job = []

for score in scores_muse:
    json_job.append({
    "in": score,
    "out": './midi\\' + score.split('\\')[-1][:-5] + '.mid',
  })

#print(json_job)

In [ ]:
import os
if 'midi' not in glob('.'):
    print('making folder')
    os.makedirs('midi')
    

In [ ]:
with open('json_job.json', 'w') as outfile:
    json.dump(json_job, outfile)

In [ ]:
! MuseScore3.exe -j json_job.json

In [ ]:
data = pd.read_csv('./metadata.tsv', sep='\t')
data.head()

In [ ]:
data.columns # to use later especially file name for merging, duration, key, composed start and composed end

In [ ]:
scores = glob('./midi/*.mid')
print(len(scores))

In [ ]:
score_names = pd.Series(scores, index=[score.split('\\')[-1].split('.')[0] for score in scores], name='filepath')
score_names.head()

In [ ]:
data = data.merge(score_names, left_on='fnames', right_index=True)
data.head()

In [ ]:
utm_max, utm_argmax, utm = most_resonant(data['filepath'][0])

In [ ]:
utm_sum = max_utm_to_ws_utm(utm_max, utm_argmax, utm)

In [ ]:
com = center_of_mass_v(utm_max, utm_argmax, utm)
print(com)

In [ ]:
Wavescape(utm_sum, 500).draw()
print(data['fnames'][0])

In [ ]:
all_wavescapes(data['filepath'][0], individual_width=500, aw_size=4, aw_per_tick=1, tick_start=1)

In [15]:
if 'figures' not in glob('.'):
    print('making folder')
    os.makedirs('figures')


making folder


In [ ]:
import matplotlib.patches as mpatches
patches = []
for i in range(6):
    patches.append(mpatches.Patch(color=np.array(circular_hue_revised(i))[:3]/255, label='Coefficient '+str(i+1)))
        
for score in data['filepath']:
    print(str(score.split('\\')[-1].split('.')[0]))
    try:
        #all_wavescapes(score, individual_width=500, aw_size=4, aw_per_tick=1, tick_start=1)
        utm_max, utm_argmax, utm = most_resonant(score, aw_size=4)
        utm_sum = max_utm_to_ws_utm(utm_max, utm_argmax, utm)
        Wavescape(utm_sum, 500).draw()
        plt.suptitle(str(score.split('\\')[-1].split('.')[0]) + '\n', fontsize=14)
        plt.legend(handles=patches)
        plt.savefig('./figures/' + str(score.split('\\')[-1].split('.')[0]) + '.png')
    except:
        print('Not successful')

In [ ]:
# legend
import webcolors

for angle in range(6):

    value = np.array(circular_hue_revised(angle))[:3]
    print('coefficient', angle+1, ':',  webcolors.rgb_to_name(value))